In [ ]:
# get catalogue
import requests
import json
source = "https://catalog-api.udacity.com/v1/catalog?locale=en-us"

r = requests.get(source)
print("fetching catalogue data")

data = r.json()

with open("./data/raw_data.json", "w") as text_file:
    text_file.write(json.dumps(data, indent=2))
    print("done writing data")

In [ ]:
import json
import requests
# ---
# Utilities
# ---

def filter_by_level(data, level):
    return [course for course in data if course["level"] == level]

def write_to_file(data, path_name):
    with open(f"{path_name}", "w+") as data_file:
        data_file.write(json.dumps(data, indent=2))
        print(f"done writing to {path_name}")

def fetch_reviews_for_key(key):
    source = f"https://ratings-api.udacity.com/api/v1/reviews?node={key}&limit=5000&page=1"
    r = requests.get(source)
    return r.json()


In [ ]:

# course jobs
with open("./data/raw_data.json") as data_file:
    data = json.load(data_file)
    
    courses = data["courses"]

    # separate beginner, intermediate and advanced courses
    beginner_courses = filter_by_level(courses, "beginner")
    intermediate_courses = filter_by_level(courses, "intermediate")
    advanced_courses = filter_by_level(courses, "advanced")

    other = [
        course["key"] for course in data["courses"] 
            if course["level"] != "beginner" and 
               course["level"] != "intermediate" and 
               course["level"] != "advanced"
    ]

    print("Start Write course jobs")

    write_to_file(beginner_courses, "./data/courses/level/beginner.json")
    write_to_file(intermediate_courses, "./data/courses/level/intermediate.json")
    write_to_file(advanced_courses, "./data/courses/level/advanced.json")

    print("Finished Write course jobs")


In [ ]:
# degree jobs
with open("./data/raw_data.json") as data_file:
    data = json.load(data_file)
    
    degrees = data["degrees"]

    beginner_degrees = filter_by_level(degrees, "beginner")
    intermediate_degrees = filter_by_level(degrees, "intermediate")
    advanced_degrees = filter_by_level(degrees, "advanced")
    
    print("Start Write degree jobs")

    write_to_file(beginner_degrees, "./data/degrees/level/beginner.json")
    write_to_file(intermediate_degrees, "./data/degrees/level/intermediate.json")
    write_to_file(advanced_degrees, "./data/degrees/level/advanced.json")
    
    print("Finished Write degree jobs")

In [ ]:
# massage data
data = fetch_reviews_for_key("nd0044")

reviews = {
  "average_rating": data["nd_avg_rating"],
  "count": data["count"],
  "stats": data["stats"]
}

print(json.dumps(reviews, indent=2))

In [ ]:
# TODO: 
# - list out all nano-degrees and courses with names and levels
# - extract all tags
# - number of projects? (does projects correlate with higher rating?)

# ---
# cleaning data
# ---
import json

with open("./data/raw_data.json") as data_file:
    json_data = json.load(data_file)
    
    degrees = [{
        "affiliates": degree.get("affiliates", ""),
        "key": degree.get("key", ""),
        "title": degree.get("title", ""),
        "level": degree.get("level", ""),
        "num_of_projects": len(degree.get("projects")) if degree.get("projects") else 0,
        "tags": degree.get("tags"),
        } for degree in json_data["degrees"] if degree["level"] != "" and degree["title"] != ""
    ]

    with open("./data/degrees.json", "w") as data_file:
        data_file.write(json.dumps(degrees))

In [3]:
import json
import requests

def fetch_reviews_for_key(key):
    source = f"https://ratings-api.udacity.com/api/v1/reviews?node={key}&limit=5000&page=1"
    r = requests.get(source)
    return r.json()

with open("./data/degrees.json", "r") as data_file:
    degrees = json.load(data_file)
    
    print("start fetching reviews")

    for degree in degrees:
        key = degree.get("key")
        data = fetch_reviews_for_key(key)
        
        degree_data = degree
        degree = {
            **degree_data,
            "average_rating": data["nd_avg_rating"],
            "count": data["count"],
            "stats": data["stats"]
            }
    
    with o"process job"pen("processed.json", "w") as output:
        output.write(json.dumps(degrees, indent=2))
        print("job has been finished")
    

start fetching reviews
job has been finished


In [ ]:
# convert processed.json to dataframe

import pandas as pd

df = pd.read_json("procesed.json")